In [1]:
from urllib.request import urlopen, urlretrieve
import geopandas as gpd
from bs4 import BeautifulSoup
import re

In [2]:
date = '01-07-2017'

## urls of shapefiles

In [3]:
url_hecto = 'https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/nwb-wegen/geogegevens/shapefile/Nederland_totaal/'+date+'/Hectopunten/'
url_max_speed = 'https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/weggeg/geogegevens/shapefile/weggeg_wegvakniveau/'+date+'/Maximum%20snelheid/'
url_inhaalverboden = 'https://www.rijkswaterstaat.nl/apps/geoservices/geodata/dmc/weggeg/geogegevens/shapefile/weggeg_wegvakniveau/'+date+'/Inhaalverboden/'

## function to download shapefiles and open them as geo dataframes

In [4]:
def get_geodf(url, feature):
    website = urlopen(url)
    soup = BeautifulSoup(website, 'lxml')
    links = soup.find_all('a', attrs={'href': re.compile(feature)})
    for link in links:
        file = link.get('href')
        urlretrieve(url+file, file)
    shapefile = feature+'.shp'
    geo_df = gpd.read_file(shapefile)
    return geo_df

## get the hectometer shapefile

In [5]:
hecto_df = get_geodf(url_hecto, 'Hectopunten')
hecto_df.head()

,AFSTAND,DVK_LETTER,HECTOMTRNG,WVK_BEGDAT,WVK_ID,ZIJDE,geometry
0,41,None,25,2001-05-01,114183002,None,POINT (57378 391777)
1,143,None,26,2001-05-01,114183002,None,POINT (57440 391858)
2,245,None,27,2001-05-01,114183002,None,POINT (57502 391938)
3,345,None,28,2001-05-01,114183002,None,POINT (57563 392018)
4,446,None,29,2001-05-01,114183002,None,POINT (57624 392098)


### update the projection

In [6]:
hecto_df.crs['towgs84'] = '565.417,50.3319,465.552,-0.398957,0.343988,-1.8774,4.0725'

In [7]:
hecto_df.crs

{'ellps': 'bessel',
 'k': 0.9999079,
 'lat_0': 52.15616055555555,
 'lon_0': 5.38763888888889,
 'no_defs': True,
 'proj': 'sterea',
 'towgs84': '565.417,50.3319,465.552,-0.398957,0.343988,-1.8774,4.0725',
 'units': 'm',
 'x_0': 155000,
 'y_0': 463000}

### change to projection to wgs84

In [8]:
hecto_df = hecto_df.to_crs(epsg=4326)

### functions to get the longitude and latitude from the points

In [9]:
def getLon(x):
    lon = x.coords.xy[0][0]
    return lon

In [10]:
def getLat(x):
    lat = x.coords.xy[1][0]
    return lat

### add longitude and latitude columns to the dataframe

In [11]:
hecto_df['LON'] = hecto_df.geometry.apply(lambda x: getLon(x))
hecto_df['LAT'] = hecto_df.geometry.apply(lambda x: getLat(x))

In [12]:
hecto_df.head()

,AFSTAND,DVK_LETTER,HECTOMTRNG,WVK_BEGDAT,WVK_ID,ZIJDE,geometry,LON,LAT
0,41,None,25,2001-05-01,114183002,None,POINT (3.981023090864618 51.50653256283582),3.981023,51.506533
1,143,None,26,2001-05-01,114183002,None,POINT (3.981893409350736 51.50727121263144),3.981893,51.507271
2,245,None,27,2001-05-01,114183002,None,POINT (3.982764033592901 51.50800086955766),3.982764,51.508001
3,345,None,28,2001-05-01,114183002,None,POINT (3.983620284828764 51.50873034697396),3.983620,51.508730
4,446,None,29,2001-05-01,114183002,None,POINT (3.984476563485286 51.50945981817575),3.984477,51.509460


## get the maximum snelheid shapefile

In [13]:
max_speed_df = get_geodf(url_max_speed, 'max_snelheden')
max_speed_df.rename(columns={'OMSCHR': 'MAX V'}, inplace=True)
max_speed_df.head()

,BEGAFSTAND,BEGINTIJD,EINDTIJD,ENDAFSTAND,FK_VELD1,FK_VELD4,IBN,IZI_SIDE,KANTCODE,MAX V,WVK_BEGDAT,WVK_ID,geometry
0,0,NaN,NaN,55,037441901420141101,KLK0004032245,I,L,T,100,2014-11-01,374419014,"LINESTRING (187237.911 509567.498, 187227.32 5..."
1,0,NaN,NaN,906,037515600720141101,KLK0004042562,I,L,T,130,2014-11-01,375156007,"LINESTRING (187742.046 378317.382, 187788 3782..."
2,0,NaN,NaN,558,037515600920141101,KLK0004043479,I,R,H,130,2014-11-01,375156009,"LINESTRING (187689.026 378302.559, 187754 3782..."
3,0,NaN,NaN,48,037604301420141101,KLK0004058823,I,L,T,130,2014-11-01,376043014,"LINESTRING (188435.397 321505.252, 188481.384 ..."
4,0,19.0,6.0,179,041934401720141101,KLK0003910540,I,R,H,130,2014-11-01,419344017,"LINESTRING (209871.068 472138.01, 210048.297 4..."


## get the inhaalverbod shapefile

In [14]:
inhaalverbod_df = get_geodf(url_inhaalverboden, 'inhaalverboden')
inhaalverbod_df.rename(columns={'OMSCHR':'VERBOD'}, inplace=True)
inhaalverbod_df.head()

,BEGAFSTAND,ENDAFSTAND,FK_VELD1,FK_VELD4,IBN,IZI_SIDE,KANTCODE,VERBOD,WVK_BEGDAT,WVK_ID,geometry
0,0,1949,040053000720141101,KLK0004303896,I,R,H,06 - 10 u. en 15 - 19 u. (vrachtauto's),2014-11-01,400530007,"LINESTRING (200260.276 565269.461, 200314 5653..."
1,0,524,042944402720141101,KLK0004622733,I,L,T,06 - 19 u. (vrachtauto's),2014-11-01,429444027,"LINESTRING (214661.624 522281.058, 214687 5222..."
2,0,459,043044400720141101,KLK0004622732,I,R,H,06 - 19 u. (vrachtauto's),2014-11-01,430444007,"LINESTRING (215166.382 522158.655, 215325 5221..."
3,0,775,043055000620141101,KLK0004542102,I,R,H,06 - 10 u. en 15 - 19 u. (vrachtauto's),2014-11-01,430550006,"LINESTRING (215324.65 575263.1949999999, 21548..."
4,0,398,012721700820141101,KLK0004510125,I,R,H,00 - 24 u. (algemeen verbod),2014-11-01,127217008,"LINESTRING (63945.835 408593.274, 64096 408606..."


## join all the dataframes on the WVK_ID column

In [15]:
df = hecto_df.merge(max_speed_df, how='left', on='WVK_ID', suffixes=('', '_ms')).merge(inhaalverbod_df, how='left', on='WVK_ID', suffixes=('', '_iv'))

## select the columns and write to csv

In [16]:
df.columns

Index(['AFSTAND', 'DVK_LETTER', 'HECTOMTRNG', 'WVK_BEGDAT', 'WVK_ID', 'ZIJDE',
       'geometry', 'LON', 'LAT', 'BEGAFSTAND', 'BEGINTIJD', 'EINDTIJD',
       'ENDAFSTAND', 'FK_VELD1', 'FK_VELD4', 'IBN', 'IZI_SIDE', 'KANTCODE',
       'MAX V', 'WVK_BEGDAT_ms', 'geometry_ms', 'BEGAFSTAND_iv',
       'ENDAFSTAND_iv', 'FK_VELD1_iv', 'FK_VELD4_iv', 'IBN_iv', 'IZI_SIDE_iv',
       'KANTCODE_iv', 'VERBOD', 'WVK_BEGDAT_iv', 'geometry_iv'],
      dtype='object')

In [17]:
cols = ['LAT', 'LON', 'DVK_LETTER', 'HECTOMTRNG', 'ZIJDE', 'MAX V', 'VERBOD']

In [18]:
df[cols].to_csv('ruudsAwesomeHectomterBestand.csv')